<a href="https://colab.research.google.com/github/proyectosRVyderivados/cristina/blob/main/ForwardETTI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install numpy-financial

#Forward $r_{12}$ conocidos dos bonos
En el mercado cotizan los bonos A y B.
* El bono A es un bono cupón cero de duración un año y TIR del 10%
* El bono B es un bono cupón explícito de 4.400 € anuales,  nominal 10.000 €, duración 2 años y precio de 14.000 €
* El bono C es un bono cupón cero a dos años, que se obtiene como bono sintético combinando los bonos A y B

Se pide:
* Calcular la TIR del bono C
* Comprobar que la TIR del bono C es el punto de la ETTI a dos años ($r_{02}$). Para hacer la comprobación calcular con la ETTI los precios de los bonos A, B y C y comprobar que coinciden con los previstos.
* Comprobar que el nominal elegido para el bono A puede ser cualquiera, es arbitrario.

In [ ]:
import numpy_financial as npf
from random import randint, seed
seed()

# DATOS
tirA = .1
cuponB = 4_400
nominalB = 10_000
precioB = 14_000
nominalA = randint(1,10000)
print(f"El nominal elegido para el bono A es {nominalA}")

# FLUJOS DE CAJA bonos A y B
bonoB = [-precioB, cuponB, cuponB+nominalB]
bonoA = [-nominalA/(1+tirA), nominalA, 0]    # añadimos un flujo 0 en t=2 por comodidad, para luego restar flujos
m = bonoB[1]/bonoA[1]
print(f"El multiplicador m es {m}")
bonoC = [bonoB[0]-m*bonoA[0], bonoB[1]-m*bonoA[1], bonoB[2]-m*bonoA[2]]
print("Bono C: ", bonoC)

# TIR del bono C
tirC = npf.irr(bonoC)
print(f"La TIR del bono C es: {tirC:.6%}")

# Creación de la ETTI
etti = [tirA, tirC] # la tirC si pertenece a la ETTI por ser un bono cupón cero, pero la tirB no pertenece
r01=etti[0]
r02=etti[1]
print(f"r01: {r01:.1%}")
print(f"r02: {r02:.1%}")
# COMPROBAR PRECIOS CON LA ETTI
def precio_etti(flujos, etti):
    precio = 0
    for i in range(1, len(flujos)):
        precio += flujos[i] * (1+etti[i-1])**-i
    return precio
# Precios de los Bonos A, B y C
precioA = precio_etti(bonoA, etti)
precioB = precio_etti(bonoB, etti)
precioC = precio_etti(bonoC, etti)
print(f"La diferencia de precios en el bono A es {precioA+bonoA[0]}")
print(f"La diferencia de precios en el bono B es {precioB+bonoB[0]}")
print(f"La diferencia de precios en el bono C es {precioC+bonoC[0]}")

El nominal elegido para el bono A es 8212
El multiplicador m es 0.535801266439357
Bono C:  [-10000.0, 0.0, 14400.0]
La TIR del bono C es: 20.000000%
r01: 10.0%
r02: 20.0%
La diferencia de precios en el bono A es 0.0
La diferencia de precios en el bono B es -3.637978807091713e-12
La diferencia de precios en el bono C es -3.637978807091713e-12


#Calcular la ETTI con una cartera de tres bonos:
### FASE 1
En el mercado cotizan los bonos A , B y C.
* El bono A es un bono cupón cero a un año de nominal 1.000 € y TIR del 10%
* El bono B es un bono cupón explícito de 90 € anuales,  nominal 1.000 € y duración 2 años
* El bono C es un bono cupón explícito de 500 € anuales, nominal 1.000 € y duración 3 años
* Conocemos la ETTI a 1, 2 y 3 años que es 10%, 20% y 30%, respectivamente  

Se pide:
* Calcular los precios de los bonos A, B y C
* Calcular la TIR de los bonos A, B y C

### FASE 2
Supongamos conocidos los flujos de caja de los bonos A, B y C incluidos sus precios.  

Se pide:
* **Calcular la ETTI de los años 1, 2 y 3.**

In [ ]:
import numpy_financial as npf

#####   FASE 1   #####
# Flujos de caja de los bonos
bonoA = [0,1000,0,0]     # inicialmente el flujo de caja inicial es cero
bonoB = [0,90,1090,0]     #pongo ya directamente los flujos de caja
bonoC = [0,500,500,1500]

# Calcular el precio de los bonos dada la ETTI
etti = [.1, .2, .3]
def precio_etti(flujos, etti):
    precio = 0
    for i in range(1, len(flujos)):
        precio += flujos[i] * (1+etti[i-1])**-i
    return precio

precioA = precio_etti(bonoA, etti)
precioB = precio_etti(bonoB, etti)
precioC = precio_etti(bonoC, etti)
print(f"El precio del bono A es {precioA:,.2f} €")
print(f"El precio del bono B es {precioB:,.2f} €")
print(f"El precio del bono C es {precioC:,.2f} €")
print()

#Calcular la TIR de los bonos:
bonoA[0] = -precioA
bonoB[0] = -precioB
bonoC[0] = -precioC

tirA = npf.irr(bonoA)
tirB = npf.irr(bonoB)
tirC = npf.irr(bonoC)

print(f"La TIR del bono A es: {tirA:.2%}")
print(f"La TIR del bono B es: {tirB:.2%}")
print(f"La TIR del bono C es: {tirC:.2%}")
print()

#####   FASE 2   #####
etti_calculada = [0,0,0]  # inicializamos el array, donde se guardaran los resultados de la ETTI.
etti_calculada[0] = tirA  # la ETTI del año 1 es la TIR del bono A por ser un bono cupón cero a un año
def calcula_etti(bono, etti_calculada):
    n = [i for i, e in enumerate(bono) if e != 0][-1] # Para calcular n
    ultimo_flujo = bono[n]
    precio = -bono[0]
    fcid = 0 # inicializamos los flujos de caja intermedios descontados
    for t in range(1,n): # recorre los flujos de caja intermedios
        fcid += bono[t] / (1+etti[t-1])**t
    return (ultimo_flujo / (precio - fcid))**(1/n)-1
r02 = calcula_etti(bonoB, etti_calculada) # calculamos r02 conocido el bono B y r01
etti_calculada[1] = r02

r03 = calcula_etti(bonoC, etti_calculada) # calculamos r03 conocido el bono C, r01 y r02
etti_calculada[2] = r03

# Imprime la etti_calculada
for i in range(len(etti_calculada)):
    print(f"La ETTI del año {i+1} es {etti_calculada[i]:.2%}")

El precio del bono A es 909.09 €
El precio del bono B es 838.76 €
El precio del bono C es 1,484.52 €

La TIR del bono A es: 10.00%
La TIR del bono B es: 19.49%
La TIR del bono C es: 25.13%

La ETTI del año 1 es 10.00%
La ETTI del año 2 es 20.00%
La ETTI del año 3 es 30.00%


# Forward implícito
En el mercado están disponibles los siguientes bonos:
* Bono A: Letra del Tesoro de duración un año y precio de adquisición 933 €
* Bono B: Bono cupón explícito del 7% y duración 2 años que se adquiere con un descuento del 2% sobre el nominal.

Se pide:
* Determinar el tipo de interés forward implícito para el periodo que se inicia en t=1 y finaliza en t=2 años, $r_{12}$.
* Calcular $r_{02}$
* Calcular nuevamente $r_{12}$ utilizando para ello el valor de $r_{02}$ anteriormente calculado
* Comprobar que el forward $r_{12}$ coincide por ambos métodos
$$ (1+r_{02})^2 = (1+r_{01}) (1+r_{12}) $$

In [4]:
import numpy_financial as npf
import numpy as np

##############       MÉTODO 1       ##############

# DATOS   
bonoA = np.array([-933, 1000, 0])
bonoB = np.array([-98, 7, 107])

# Bono C
bonoC = np.array([0,0,0])        # inicializamos el ndarray
bonoC = bonoA[0]*bonoB - bonoB[0]*bonoA
print(f"Bono C: {bonoC}")
# Forward r12
r12 = npf.irr(bonoC)
print(f"MÉTODO 1: el forward r12 es {r12:.4%}")

##############       MÉTODO 2       ##############

# Bono D
bonoD = np.array([0,0,0])        # inicializamos el ndarray
bonoD = bonoA[1]*bonoB - bonoB[1]*bonoA
print(f"Bono D: {bonoD}")

# ETTI a dos años r02
r02 = npf.irr(bonoD)
print(f"La ETTI a dos años es r02 = {r02:.4%}")

# TIR del Bono A (r01)
r01 = npf.irr(bonoA)
print(f"La TIR del bono A es r01 = {r01:.4%}")

##############      COMPROBACIÓN      ##############

# Comprobación r12 = r12_bis
r12_bis = (1+r02)**2 / (1+r01) -1
print(f"MÉTODO 2: el forward r12 es {r12_bis:.4%}")
print(f"La diferencia de r12 por ambos métodos es {abs(r12 - r12_bis)}")

Bono C: [     0  91469 -99831]
MÉTODO 1: el forward r12 es 9.1419%
Bono D: [-91469      0 107000]
La ETTI a dos años es r02 = 8.1571%
La TIR del bono A es r01 = 7.1811%
MÉTODO 2: el forward r12 es 9.1419%
La diferencia de r12 por ambos métodos es 0.0


# Cálculo de un tipo Forward $r_{23}$
En un mercado de renta fija cotizan los siguienes bonos:
* Bono A: es un bono cupón cero a un año que se adquiere por 100 y se amortiza por 110
* Bono B: es un bono cupón cero a dos años con TIR del 9% y precio de adquisición de 500 €
* Bono C: es un bono cupón cero a tres años con TIR del 8% y nominal de 1.000 €  

Se pide:
* Calcular el tipo de interés forward implícito en el mercado: $r_{23}$

In [6]:
import numpy_financial as npf
import numpy as np

# DATOS   
bonoA = np.array([-100, 110, 0, 0])
bonoB = np.array([-500, 0, 500*1.09**2, 0])  # la TIR del bono B es del 9%
bonoC = np.array([-1000/1.08**3, 0, 0, 1000])

print(f"Bono A: {bonoA}")
print(f"Bono B: {bonoB}")
print(f"Bono C: {bonoC}")

##############       MÉTODO 1       ##############

bonoD = np.ndarray([0,0,0,0])
bonoD = -bonoC[0] * bonoB + bonoB[0] * bonoC
print(f"Bono D: {bonoD}")
tirD = npf.irr(bonoD)
print(f"Método 1: el forward r23 es {tirD:.5%}")

##############       MÉTODO 2       ##############
r23 = (1+0.08)**3 / (1+0.09)**2 -1
print(f"Método 2: el forward r23 es {r23:.5%}")

Bono A: [-100  110    0    0]
Bono B: [-500.      0.    594.05    0.  ]
Bono C: [-793.83224102    0.            0.         1000.        ]
Bono D: [      0.               0.          471576.04277803 -500000.        ]
Método 1: el forward r23 es 6.02744%
Método 2: el forward r23 es 6.02744%
